# RAG Query Pipeline
## 1. Imports thư viện cần thiết

In [2]:
# pip install langchain-
import os
import chromadb
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print("✅ Đã import thành công các thư viện cần thiết cho RAG pipeline.")

✅ Đã import thành công các thư viện cần thiết cho RAG pipeline.


## 2. Khởi tạo Mô hình (LLM và Embedding)

In [13]:
# Cấu hình biến môi trường cho Groq API Key trong file .env
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
print("✅ Đã cấu hình biến môi trường GROQ_API_KEY từ file .env.")

model = "groq/compound"
# 1. Khởi tạo LLM (Siêu nhanh và miễn phí từ Groq)
# Dùng model "groq/compound"
llm = ChatGroq(
    model= model,
    # temperature là tham số điều chỉnh độ sáng tạo của câu trả lời
    temperature=0 # Gần như = 0 để câu trả lời bám sát context
)
print(f"✅ Đã khởi tạo LLM (Groq Compound).")

# 2. Khởi tạo Embedding Model (Dùng lại model cũ)
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print("✅ Đã khởi tạo Embedding Model (multilingual-MiniLM).")

✅ Đã cấu hình biến môi trường GROQ_API_KEY từ file .env.
✅ Đã khởi tạo LLM (Groq Compound).
✅ Đã khởi tạo Embedding Model (multilingual-MiniLM).


## 3. Tải Vector Store

In [14]:
# Tải lại database đã được xây dựng và LỌC SẠCH ở file 01
db_path = "./chroma_db"
collection_name = "academic_regulations" # Đảm bảo tên này khớp với file 01

vectorstore = Chroma(
    persist_directory=db_path,
    embedding_function=embedding_model,
    collection_name=collection_name
)

# Tạo một "retriever" để lấy tài liệu
# search_kwargs={"k": 5}: Lấy 5 chunk liên quan nhất
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

print(f"✅ Đã tải thành công ChromaDB từ '{db_path}' (collection: '{collection_name}')")
print(f"📚 Số lượng chunks trong DB: {vectorstore._collection.count()}")

✅ Đã tải thành công ChromaDB từ './chroma_db' (collection: 'academic_regulations')
📚 Số lượng chunks trong DB: 156


## 4. Tạo Chuỗi RAG Hoàn chỉnh (End-to-End)

In [15]:
# 1. Tạo prompt RAG chính
# Ra lệnh cho LLM trả lời DỰA TRÊN context được cung cấp
rag_template = """
Bạn là một trợ lý AI hữu ích, chuyên trả lời các câu hỏi về quy định học thuật của trường 
ĐH Sư Phạm Kỹ Thuật TP.HCM dựa trên các văn bản được cung cấp.
Hãy trả lời câu hỏi của người dùng một cách ngắn gọn và chính xác, 
chỉ dựa vào nội dung trong phần "Văn bản tham khảo" dưới đây.
KHÔNG được bịa đặt thông tin. Nếu không tìm thấy, hãy nói "Tôi không tìm thấy thông tin này trong tài liệu."

Văn bản tham khảo:
{context}

Câu hỏi:
{question}

Câu trả lời (chỉ dựa trên văn bản):
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

# 2. Hàm này để gộp các chunk tìm được thành 1 đoạn text
def format_context(docs):
    return "\n\n---\n\n".join([d.page_content for d in docs])

# 3. TẠO CHUỖI RAG HOÀN CHỈNH
# Đây là "công thức" của RAG:
# 1. Lấy câu hỏi gốc (RunnablePassthrough)
# 2. Dùng câu đã mở rộng để tìm context (retriever)
# 3. Gộp context lại (format_context)
# 4. Nhồi context và câu hỏi gốc vào RAG prompt (rag_prompt)
# 5. Đưa cho LLM trả lời (llm)
# 6. Lấy kết quả (StrOutputParser)

rag_chain = (
    {
        "context": RunnablePassthrough() 
                   | retriever 
                   | format_context,
        "question": RunnablePassthrough()
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("✅ Đã tạo chuỗi RAG hoàn chỉnh.")

✅ Đã tạo chuỗi RAG hoàn chỉnh.


In [16]:
import requests
import os

api_key = os.environ.get("GROQ_API_KEY")
url = "https://api.groq.com/openai/v1/models"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

print(response.json())

{'object': 'list', 'data': [{'id': 'qwen/qwen3-32b', 'object': 'model', 'created': 1748396646, 'owned_by': 'Alibaba Cloud', 'active': True, 'context_window': 131072, 'public_apps': None, 'max_completion_tokens': 40960}, {'id': 'openai/gpt-oss-20b', 'object': 'model', 'created': 1754407957, 'owned_by': 'OpenAI', 'active': True, 'context_window': 131072, 'public_apps': None, 'max_completion_tokens': 65536}, {'id': 'llama-3.3-70b-versatile', 'object': 'model', 'created': 1733447754, 'owned_by': 'Meta', 'active': True, 'context_window': 131072, 'public_apps': None, 'max_completion_tokens': 32768}, {'id': 'openai/gpt-oss-safeguard-20b', 'object': 'model', 'created': 1761708789, 'owned_by': 'OpenAI', 'active': True, 'context_window': 131072, 'public_apps': None, 'max_completion_tokens': 65536}, {'id': 'moonshotai/kimi-k2-instruct', 'object': 'model', 'created': 1752435491, 'owned_by': 'Moonshot AI', 'active': True, 'context_window': 131072, 'public_apps': None, 'max_completion_tokens': 16384

## 5. 

In [39]:
# Đặt câu hỏi (viết tắt)
user_query = "thời gian đkmh là khi nào?"

print(f"🔍 Đang thực hiện truy vấn RAG cho câu hỏi: '{user_query}'")
print("\n" + "="*80)

# Dùng .stream() để xem LLM "gõ" câu trả lời
full_response = ""

for chunk in rag_chain.stream(user_query):
    print(chunk, end="", flush=True)
    full_response += chunk

print("\n" + "="*80)
print(f"\n📄 Câu trả lời đầy đủ:\n{full_response}")
print("\n✅ Hoàn tất!")

🔍 Đang thực hiện truy vấn RAG cho câu hỏi: 'thời gian đkmh là khi nào?'

**Thời gian đăng ký môn học (đkmh) trong học kỳ:**

- **Đợt 1:** từ ngày **11 / 08 / 2025** đến hết ngày **23 / 08 / 2025**.  
- **Đợt 2:** từ ngày **27 / 10 / 2025** đến hết ngày **01 / 11 / 2025**.

📄 Câu trả lời đầy đủ:
**Thời gian đăng ký môn học (đkmh) trong học kỳ:**

- **Đợt 1:** từ ngày **11 / 08 / 2025** đến hết ngày **23 / 08 / 2025**.  
- **Đợt 2:** từ ngày **27 / 10 / 2025** đến hết ngày **01 / 11 / 2025**.

✅ Hoàn tất!
